<a href="https://colab.research.google.com/github/KevinGJ23/Informatica_medica_2025/blob/main/modelo_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Importacion**

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 14.7 MB/s eta 0:00:00


In [ ]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 9.1 MB/s eta 0:00:00


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.7 MB/s eta 0:00:00


##**Librerias**

In [ ]:
#Importacion de librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from category_encoders import CatBoostEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import optuna
pd.set_option('display.max_columns', None)

##**Funciones**

In [ ]:
#Funcion resumen de datos
def resumen(dataframe):
  # Cálculos de resumen
  cantidad_nulos= dataframe.isnull().sum()
  cantidad_total = len(dataframe)
  cantidad_no_nulos = dataframe.notnull().sum()
  tipo_de_datos = dataframe.dtypes
  cantidad_de_clases = dataframe.nunique()
  # Crear DataFrame de resumen
  resumen_df = pd.DataFrame({
      'Tipo de Dato': tipo_de_datos,
      'Valores Únicos': cantidad_de_clases,
      'No Nulos': cantidad_no_nulos,
      'Nulos': cantidad_nulos,
      '% Nulos': (cantidad_nulos / cantidad_total) * 100
  })
  # Ordenar por % de nulos
  resumen_df = resumen_df.sort_values(by='% Nulos', ascending=False)
  # Aplicar formato con Styler
  tabla_resumen = resumen_df.style \
      .format({'% Nulos': '{:.2f}%'}) \
      .bar(subset='% Nulos', color='#f08080') \
      .set_properties(**{'text-align': 'center'}) \
      .set_caption("Resumen de Columnas del DataFrame") \
      .set_table_styles([
          # Color del encabezado de columnas
          {'selector': 'th', 'props': [
              ('text-align', 'center'),
              ('background-color', '#1f4e79'),
              ('color', 'white'),
              ('font-weight', 'bold')
          ]},
          # Color de las celdas
          {'selector': 'td', 'props': [('padding', '5px')]},
          # Estilo del título de la tabla
          {'selector': 'caption', 'props': [
              ('color', '#1f4e79'),
              ('font-size', '18px'),
              ('text-align', 'center'),
              ('font-weight', 'bold')
          ]}
      ])
  # Mostrar
  display(tabla_resumen)
  # Información final
  print(f"\n🧾 El DataFrame tiene un total de **{len(dataframe.columns)}** columnas y **{len(dataframe)}** filas.")

In [ ]:
#Función Encabezado
def encabezado(dataframe,filas=5):
  salida=dataframe.head(filas).style \
      .set_properties(**{
          'background-color': '#f9f9f9',
          'border': '1px solid #ddd',
          'text-align': 'center'
      }) \
      .set_table_styles([
          {'selector': 'th', 'props': [('background-color', '#4B8BBE'), ('color', 'white'), ('text-align', 'center')]}
      ]) \
      .highlight_null() \
      .set_caption("🔍 Vista Previa de los Primeros Registros")
  display(salida)

In [ ]:
X=pd.read_csv('X.csv')
y=pd.read_csv('y.csv')

##**Visualizacion**

In [ ]:
y=y.drop(columns=['GRD_AGRUPADO_2','GRD_GRUPO_2DIG','GRD_AGRUPADO'])

In [ ]:
X=X.drop(columns=['PROC_12_GRUPO','PROC_13_GRUPO','PROC_11_GRUPO','PROC_10_GRUPO','PROC_9_GRUPO','PROC_8_GRUPO',
                        'PROC_7_GRUPO','PROC_6_GRUPO','PROC_5_GRUPO','PROC_4_GRUPO','MES','DIA_SEMANA','INFECCIONES',
                        'INFECCION_QUIRURGICA','DXR_9','DXR_8','DXR_7','DXR_6','DXR_5','DXR_4','DXR_2'])

In [ ]:
encabezado(X)

,SEXO,TIPO_INGRESO,CUIDADOS_INTENSIVOS,DXR_1,DXR_3,SITUACION_ALTA,TIPO_SERVICIO,DX_EGRESO_AGRUPADO,PROC_1_GRUPO,PROC_2_GRUPO,PROC_3_GRUPO,EDAD_AGRUPADA,ESTANCIA_AGRUPADA,UCI_AGRUPADA
0,F,URGENCIA,NO,Circulatorio,Neoplasias,ALTA MÉDICA,URGENCIA ADULTOS,Circulatorio,Genitourinario,Genitourinario,Diagnóstico/terapéutico,Joven adulto,15+ días,Sin cuidados
1,M,URGENCIA,NO,Infecciosas,Infecciosas,ALTA MÉDICA,NO APLICA,Infecciosas,Diagnóstico/terapéutico,Desconocido,Desconocido,Joven adulto,15+ días,Sin cuidados
2,M,URGENCIA,NO,Infecciosas,Infecciosas,ALTA MÉDICA,URGENCIA ADULTOS,Infecciosas,Digestivo,Genitourinario,Diagnóstico/terapéutico,Joven adulto,15+ días,Sin cuidados
3,M,URGENCIA,NO,Infecciosas,Infecciosas,ALTA MÉDICA,URGENCIA ADULTOS,Infecciosas,Diagnóstico/terapéutico,Desconocido,Desconocido,Joven adulto,2-4 días,Sin cuidados
4,M,URGENCIA,NO,Traumatismo,Genitourinario,ALTA MÉDICA,URGENCIA ADULTOS,Traumatismo,Genitourinario,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Adulto,15+ días,Sin cuidados


In [ ]:
encabezado(y)

,GRD_ORIGINAL,CODIGOS_SOLOS
0,14143,OTROS
1,184153,OTROS
2,184153,OTROS
3,184153,OTROS
4,184112,OTROS


In [ ]:
resumen(y)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/style.py:4040: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
GRD_ORIGINAL,int64,608,27869,0,0.00%
CODIGOS_SOLOS,object,107,27869,0,0.00%



🧾 El DataFrame tiene un total de **2** columnas y **27869** filas.


In [ ]:
resumen(X)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/style.py:4040: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
SEXO,object,2,27869,0,0.00%
TIPO_INGRESO,object,2,27869,0,0.00%
CUIDADOS_INTENSIVOS,object,2,27869,0,0.00%
DXR_1,object,15,27869,0,0.00%
DXR_3,object,15,27869,0,0.00%
SITUACION_ALTA,object,2,27869,0,0.00%
TIPO_SERVICIO,object,4,27869,0,0.00%
DX_EGRESO_AGRUPADO,object,14,27869,0,0.00%
PROC_1_GRUPO,object,17,27869,0,0.00%
PROC_2_GRUPO,object,16,27869,0,0.00%



🧾 El DataFrame tiene un total de **14** columnas y **27869** filas.


##**Codificación**

###**Importancia Variables**

In [ ]:
ordinal_diagnosticos={
    'Vacio':0,
    'Infecciosas': 1,
    'Neoplasias': 2,
    'Endocrino': 3,
    'Psicológico': 4,
    'Neurológico': 5,
    'Sensorial': 6,
    'Circulatorio': 7,
    'Respiratorio': 8,
    'Digestivo': 9,
    'Dermatológico': 10,
    'Musculoesquelético': 11,
    'Genitourinario': 12,
    'Traumatismo': 13,
    'Otros':14
}
ordinal_procesos={
    'Desconocido':0,
    'Misceláneos': 1,
    'Sistema nervioso': 2,
    'Endócrino': 3,
    'Ojo y oído': 4,
    'Respiratorio': 5,
    'Cardiovascular': 6,
    'Linfático y bazo': 7,
    'Digestivo': 8,
    'Genitourinario': 9,
    'Parto y puerperio': 10,
    'Musculoesquelético': 11,
    'Tegumentario': 12,
    'Obstetricia': 13,
    'Cirugía músculo-esquelética': 14,
    'Mama y piel': 15,
    'Diagnóstico/terapéutico': 16,
    'Otros':17
}

ordinal_edad={
    'Neonato':0,
    'Niñez':1,
    'Adolescencia':2,
    'Joven adulto':3,
    'Adulto':4,
    'Adulto mayor':5
}

ordinal_dia={
    'Lunes':0,
    'Martes':1,
    'Miércoles':2,
    'Jueves':3,
    'Viernes':4,
    'Sábado':5,
    'Domingo':6
}
ordinal_dias_estancia={
    '<2 días':0,
    '2-4 días':1,
    '5-7 días':2,
    '8-14 días':3,
    '15+ días':4
}
ordinal_uci={
    'Sin cuidados':0,
    '1 día':1,
    '2-4 días':2,
    '5-9 días':3,
    '10+ días':4
}
ordinal_tipo_servicio={
    'CIRUGÍA':0,
    'URGENCIA PEDÍATRICAS':1,
    'NO APLICA':2,
    'URGENCIA ADULTOS':3
}


In [ ]:
X_nueva=X.copy()
# Diagnósticos (puede haber varios)
for col in ['DXR_1', 'DXR_3','DX_EGRESO_AGRUPADO']:
    if col in X_nueva.columns:
        X_nueva[col] = X_nueva[col].map(ordinal_diagnosticos)

# Procesos
for col in ['PROC_1_GRUPO', 'PROC_2_GRUPO', 'PROC_3_GRUPO']:
    if col in X_nueva.columns:
        X_nueva[col] = X_nueva[col].map(ordinal_procesos)

# Edad
if 'EDAD_AGRUPADA' in X_nueva.columns:
    X_nueva['EDAD_AGRUPADA'] = X_nueva['EDAD_AGRUPADA'].map(ordinal_edad)

# Estancia
if 'ESTANCIA_AGRUPADA' in X_nueva.columns:
    X_nueva['ESTANCIA_AGRUPADA'] = X_nueva['ESTANCIA_AGRUPADA'].map(ordinal_dias_estancia)

# UCI
if 'UCI_AGRUPADA' in X_nueva.columns:
    X_nueva['UCI_AGRUPADA'] = X_nueva['UCI_AGRUPADA'].map(ordinal_uci)

# Tipo de servicio
if 'TIPO_SERVICIO' in X_nueva.columns:
    X_nueva['TIPO_SERVICIO'] = X_nueva['TIPO_SERVICIO'].map(ordinal_tipo_servicio)

# Mapear manualmente a 0 y 1 (ajusta según tus valores reales)

X_nueva['SEXO'] = X_nueva['SEXO'].map({'F': 0, 'M': 1})
X_nueva['CUIDADOS_INTENSIVOS'] = X_nueva['CUIDADOS_INTENSIVOS'].map({'NO': 0, 'SI': 1})
X_nueva['TIPO_INGRESO'] = X_nueva['TIPO_INGRESO'].map({'PROGRAMADO': 0, 'URGENCIA': 1})
X_nueva['SITUACION_ALTA'] = X_nueva['SITUACION_ALTA'].map({'ALTA MÉDICA': 0, 'FALLECIDO': 1})



In [ ]:
encabezado(X_nueva)

,SEXO,TIPO_INGRESO,CUIDADOS_INTENSIVOS,DXR_1,DXR_3,SITUACION_ALTA,TIPO_SERVICIO,DX_EGRESO_AGRUPADO,PROC_1_GRUPO,PROC_2_GRUPO,PROC_3_GRUPO,EDAD_AGRUPADA,ESTANCIA_AGRUPADA,UCI_AGRUPADA
0,0,1,0,7,2,0,3,7,9,9,16,3,4,0
1,1,1,0,1,1,0,2,1,16,0,0,3,4,0
2,1,1,0,1,1,0,3,1,8,9,16,3,4,0
3,1,1,0,1,1,0,3,1,16,0,0,3,1,0
4,1,1,0,13,12,0,3,13,9,16,16,4,4,0


In [ ]:
resumen(X_nueva)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/style.py:4040: RuntimeWarning: invalid value encountered in scalar divide
  end = (x - left) / (right - left)


,Tipo de Dato,Valores Únicos,No Nulos,Nulos,% Nulos
SEXO,int64,2,27869,0,0.00%
TIPO_INGRESO,int64,2,27869,0,0.00%
CUIDADOS_INTENSIVOS,int64,2,27869,0,0.00%
DXR_1,int64,15,27869,0,0.00%
DXR_3,int64,15,27869,0,0.00%
SITUACION_ALTA,int64,2,27869,0,0.00%
TIPO_SERVICIO,int64,4,27869,0,0.00%
DX_EGRESO_AGRUPADO,int64,14,27869,0,0.00%
PROC_1_GRUPO,int64,17,27869,0,0.00%
PROC_2_GRUPO,int64,16,27869,0,0.00%



🧾 El DataFrame tiene un total de **14** columnas y **27869** filas.


In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_nueva, y.CODIGOS_SOLOS)


In [ ]:
# Separar los datos
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, stratify=y_res, test_size=0.2, random_state=42)

In [ ]:
y_train=np.array(y_train)
y_test=np.array(y_test)

##**Modelo**

In [ ]:
def objective(trial):
    params = {
        'loss_function': 'MultiClass',
        'eval_metric': 'MultiClass',
        'task_type': 'GPU',
        'random_seed': 42,
        'verbose': 0,

        'iterations': trial.suggest_int('iterations', 300, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.2, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 5.0, log=True),
        'border_count': trial.suggest_int('border_count', 32, 128),
        'random_strength': trial.suggest_float('random_strength', 1e-3, 5.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise']),
    }

    model = CatBoostClassifier(**params)

    model.fit(
        X_train, y_train,
        eval_set=(X_test, y_test),
        early_stopping_rounds=30,
        verbose=False
    )

    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    return acc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15, timeout=600)  # ligero


# 4. Resultados
print("🏆 Mejor accuracy en validación:", study.best_value)
print("🧪 Mejores hiperparámetros encontrados:")
print(study.best_params)

# 5. Entrenar el mejor modelo
best_model = CatBoostClassifier(**study.best_params,
                                loss_function='MultiClass',
                                task_type='GPU',
                                eval_metric='MultiClass',
                                random_seed=42)

best_model.fit(X_train, y_train)

# 6. Evaluar
test_preds = best_model.predict(X_test)
test_acc = accuracy_score(y_test, test_preds)
print(f"🎯 Accuracy en test: {test_acc:.4f}")

# 7. Métricas adicionales (opcional pero recomendado)
print("\n📊 Reporte de clasificación:")
print(classification_report(y_test, test_preds))

print("🧩 Matriz de confusión:")
print(confusion_matrix(y_test, test_preds))

[I 2025-07-20 18:34:20,280] A new study created in memory with name: no-name-73b56aca-f456-409b-a789-4ba9c17e5ad2
[I 2025-07-20 18:39:49,598] Trial 0 finished with value: 0.6466114562384947 and parameters: {'iterations': 452, 'depth': 4, 'learning_rate': 0.1255402730170356, 'l2_leaf_reg': 1.953791467135062, 'border_count': 72, 'random_strength': 3.160544086881151, 'bagging_temperature': 0.5392218411677827, 'grow_policy': 'SymmetricTree'}. Best is trial 0 with value: 0.6466114562384947.
[I 2025-07-20 18:53:05,350] Trial 1 finished with value: 0.809416528192702 and parameters: {'iterations': 954, 'depth': 7, 'learning_rate': 0.08728965333639793, 'l2_leaf_reg': 1.4851531365952653, 'border_count': 99, 'random_strength': 0.06152849480468384, 'bagging_temperature': 0.42534108874888954, 'grow_policy': 'Depthwise'}. Best is trial 1 with value: 0.809416528192702.


🏆 Mejor accuracy en validación: 0.809416528192702
🧪 Mejores hiperparámetros encontrados:
{'iterations': 954, 'depth': 7, 'learning_rate': 0.08728965333639793, 'l2_leaf_reg': 1.4851531365952653, 'border_count': 99, 'random_strength': 0.06152849480468384, 'bagging_temperature': 0.42534108874888954, 'grow_policy': 'Depthwise'}
0:	learn: 3.5993787	total: 868ms	remaining: 13m 47s
1:	learn: 3.1428431	total: 1.82s	remaining: 14m 24s
2:	learn: 2.9266553	total: 2.68s	remaining: 14m 10s
3:	learn: 2.7796107	total: 3.5s	remaining: 13m 52s
4:	learn: 2.6597926	total: 4.32s	remaining: 13m 40s
5:	learn: 2.5594059	total: 5.14s	remaining: 13m 32s
6:	learn: 2.4761957	total: 5.96s	remaining: 13m 26s
7:	learn: 2.4010130	total: 6.79s	remaining: 13m 22s
8:	learn: 2.3368093	total: 7.61s	remaining: 13m 18s
9:	learn: 2.2811572	total: 8.43s	remaining: 13m 15s
10:	learn: 2.2302983	total: 9.24s	remaining: 13m 12s
11:	learn: 2.1874955	total: 10.1s	remaining: 13m 9s
12:	learn: 2.1472644	total: 10.9s	remaining: 13m 7

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class CodificadorPersonalizado(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.ordinal_diagnosticos = {
            'Vacio':0, 'Infecciosas': 1, 'Neoplasias': 2, 'Endocrino': 3,
            'Psicológico': 4, 'Neurológico': 5, 'Sensorial': 6, 'Circulatorio': 7,
            'Respiratorio': 8, 'Digestivo': 9, 'Dermatológico': 10,
            'Musculoesquelético': 11, 'Genitourinario': 12, 'Traumatismo': 13, 'Otros':14
        }

        self.ordinal_procesos = {
            'Desconocido':0, 'Misceláneos': 1, 'Sistema nervioso': 2, 'Endócrino': 3,
            'Ojo y oído': 4, 'Respiratorio': 5, 'Cardiovascular': 6, 'Linfático y bazo': 7,
            'Digestivo': 8, 'Genitourinario': 9, 'Parto y puerperio': 10, 'Musculoesquelético': 11,
            'Tegumentario': 12, 'Obstetricia': 13, 'Cirugía músculo-esquelética': 14,
            'Mama y piel': 15, 'Diagnóstico/terapéutico': 16, 'Otros':17
        }

        self.ordinal_edad = {'Neonato':0,'Niñez':1,'Adolescencia':2,'Joven adulto':3,'Adulto':4,'Adulto mayor':5}
        self.ordinal_dia = {'Lunes':0, 'Martes':1, 'Miércoles':2, 'Jueves':3, 'Viernes':4, 'Sábado':5, 'Domingo':6}
        self.ordinal_dias_estancia = {'<2 días':0, '2-4 días':1, '5-7 días':2, '8-14 días':3, '15+ días':4}
        self.ordinal_uci = {'Sin cuidados':0, '1 día':1, '2-4 días':2, '5-9 días':3, '10+ días':4}
        self.ordinal_tipo_servicio = {'CIRUGÍA':0, 'URGENCIA PEDÍATRICAS':1, 'NO APLICA':2, 'URGENCIA ADULTOS':3}

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_ = X.copy()

        for col in ['DXR_1','DXR_2','DXR_3','DXR_4','DXR_5','DXR_6','DXR_7','DXR_8','DXR_9','DX_EGRESO_AGRUPADO']:
            if col in X_.columns:
                X_[col] = X_[col].map(self.ordinal_diagnosticos)

        for col in ['PROC_1_GRUPO','PROC_2_GRUPO','PROC_3_GRUPO','PROC_4_GRUPO','PROC_5_GRUPO','PROC_6_GRUPO',
                    'PROC_7_GRUPO','PROC_8_GRUPO','PROC_9_GRUPO','PROC_10_GRUPO','PROC_11_GRUPO','PROC_12_GRUPO','PROC_13_GRUPO']:
            if col in X_.columns:
                X_[col] = X_[col].map(self.ordinal_procesos)

        if 'EDAD_AGRUPADA' in X_.columns:
            X_['EDAD_AGRUPADA'] = X_['EDAD_AGRUPADA'].map(self.ordinal_edad)

        if 'DIA_SEMANA' in X_.columns:
            X_['DIA_SEMANA'] = X_['DIA_SEMANA'].map(self.ordinal_dia)

        if 'ESTANCIA_AGRUPADA' in X_.columns:
            X_['ESTANCIA_AGRUPADA'] = X_['ESTANCIA_AGRUPADA'].map(self.ordinal_dias_estancia)

        if 'UCI_AGRUPADA' in X_.columns:
            X_['UCI_AGRUPADA'] = X_['UCI_AGRUPADA'].map(self.ordinal_uci)

        if 'TIPO_SERVICIO' in X_.columns:
            X_['TIPO_SERVICIO'] = X_['TIPO_SERVICIO'].map(self.ordinal_tipo_servicio)

        bin_map = {'NO': 0, 'SI': 1, 'F': 0, 'M': 1, 'PROGRAMADO': 0, 'URGENCIA': 1, 'ALTA MÉDICA': 0, 'FALLECIDO': 1}
        for col in ['SEXO','CUIDADOS_INTENSIVOS','INFECCIONES','INFECCION_QUIRURGICA','TIPO_INGRESO','SITUACION_ALTA']:
            if col in X_.columns:
                X_[col] = X_[col].map(bin_map)

        return X_


In [ ]:
from sklearn.pipeline import Pipeline

pipeline_modelo = Pipeline([
    ('codificador', CodificadorPersonalizado()),
    ('modelo', best_model)
])


In [ ]:
import joblib

joblib.dump(pipeline_modelo, 'Solenoide.pkl')


['Solenoide.pkl']

In [ ]:
modelo_cargado = joblib.load('Solenoide.pkl')

In [ ]:
X.iloc[0:50]

,SEXO,TIPO_INGRESO,CUIDADOS_INTENSIVOS,DXR_1,DXR_3,SITUACION_ALTA,TIPO_SERVICIO,DX_EGRESO_AGRUPADO,PROC_1_GRUPO,PROC_2_GRUPO,PROC_3_GRUPO,EDAD_AGRUPADA,ESTANCIA_AGRUPADA,UCI_AGRUPADA
0,F,URGENCIA,NO,Circulatorio,Neoplasias,ALTA MÉDICA,URGENCIA ADULTOS,Circulatorio,Genitourinario,Genitourinario,Diagnóstico/terapéutico,Joven adulto,15+ días,Sin cuidados
1,M,URGENCIA,NO,Infecciosas,Infecciosas,ALTA MÉDICA,NO APLICA,Infecciosas,Diagnóstico/terapéutico,Desconocido,Desconocido,Joven adulto,15+ días,Sin cuidados
2,M,URGENCIA,NO,Infecciosas,Infecciosas,ALTA MÉDICA,URGENCIA ADULTOS,Infecciosas,Digestivo,Genitourinario,Diagnóstico/terapéutico,Joven adulto,15+ días,Sin cuidados
3,M,URGENCIA,NO,Infecciosas,Infecciosas,ALTA MÉDICA,URGENCIA ADULTOS,Infecciosas,Diagnóstico/terapéutico,Desconocido,Desconocido,Joven adulto,2-4 días,Sin cuidados
4,M,URGENCIA,NO,Traumatismo,Genitourinario,ALTA MÉDICA,URGENCIA ADULTOS,Traumatismo,Genitourinario,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Adulto,15+ días,Sin cuidados
5,M,URGENCIA,SI,Circulatorio,Traumatismo,FALLECIDO,URGENCIA ADULTOS,Circulatorio,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Adulto mayor,8-14 días,2-4 días
6,F,URGENCIA,NO,Circulatorio,Digestivo,ALTA MÉDICA,URGENCIA ADULTOS,Digestivo,Genitourinario,Genitourinario,Diagnóstico/terapéutico,Adulto,8-14 días,Sin cuidados
7,M,URGENCIA,NO,Traumatismo,Genitourinario,ALTA MÉDICA,URGENCIA ADULTOS,Traumatismo,Cirugía músculo-esquelética,Cirugía músculo-esquelética,Diagnóstico/terapéutico,Adulto,15+ días,Sin cuidados
8,M,URGENCIA,NO,Neoplasias,Neoplasias,ALTA MÉDICA,URGENCIA ADULTOS,Infecciosas,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Diagnóstico/terapéutico,Adulto,15+ días,Sin cuidados
9,M,URGENCIA,NO,Digestivo,Traumatismo,ALTA MÉDICA,URGENCIA ADULTOS,Digestivo,Genitourinario,Digestivo,Genitourinario,Adulto mayor,15+ días,Sin cuidados


In [ ]:
y.CODIGOS_SOLOS[0:50]

,CODIGOS_SOLOS
0,OTROS
1,OTROS
2,OTROS
3,OTROS
4,OTROS
5,054123
6,074133
7,OTROS
8,OTROS
9,OTROS


In [ ]:
predicciones = modelo_cargado.predict(X.iloc[0:50])

In [ ]:
print(predicciones)

[['OTROS']
 ['OTROS']
 ['OTROS']
 ['064171']
 ['OTROS']
 ['054123']
 ['074133']
 ['081203']
 ['OTROS']
 ['OTROS']
 ['044213']
 ['OTROS']
 ['174133']
 ['011103']
 ['214123']
 ['174133']
 ['051203']
 ['174133']
 ['OTROS']
 ['114103']
 ['014242']
 ['044162']
 ['054232']
 ['214122']
 ['051203']
 ['011103']
 ['OTROS']
 ['081203']
 ['214123']
 ['044153']
 ['064173']
 ['031203']
 ['064103']
 ['061203']
 ['044152']
 ['031203']
 ['064133']
 ['044163']
 ['051403']
 ['OTROS']
 ['074103']
 ['031203']
 ['011103']
 ['031203']
 ['014263']
 ['091303']
 ['114103']
 ['OTROS']
 ['014222']
 ['064183']]


In [ ]:
# Asegurar que predicciones es una lista de valores planos
predichos_planos = [p[0] for p in predicciones]  # extrae la clase predicha de cada sublista
reales = y.CODIGOS_SOLOS[0:20]

# Calcular precisión
aciertos = sum([p == r for p, r in zip(predichos_planos, reales)])
media_aciertos = aciertos / len(reales)

print("Media de aciertos:", media_aciertos)

Media de aciertos: 0.75
